In [ ]:
import requests
from bs4 import BeautifulSoup

# Définir les en-têtes HTTP pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# Fonction pour scraper une page et récupérer tous les articles
def scrape_all_page(url):
    all_articles = []  # Liste de tous les articles
    seen_articles = set()  # Ensemble pour garder trace des articles déjà ajoutés

    try:
        # Envoyer une requête HTTP pour obtenir la page
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Trouver tous les articles dans les balises <div class="thread">
            articles = soup.find_all("div", class_="thread")
            
            for article in articles:
                # Extraire le titre
                title_element = article.find("h3", class_="teaser__title")
                title_text = title_element.get_text(strip=True) if title_element else "Titre non disponible"
                
                # Extraire l'URL de l'article
                link_element = article.find("a", class_="teaser__link")
                article_url = link_element["href"] if link_element else None
                if article_url and not article_url.startswith("http"):
                    article_url = f"https://www.lemonde.fr{article_url}"  # Construire l'URL complète si nécessaire
                
                # Extraire la description
                description_element = article.find("p", class_="teaser__desc")
                description_text = description_element.get_text(strip=True) if description_element else "Description non disponible"
                
                # Ajouter l'article à la liste si non déjà vu
                if article_url and article_url not in seen_articles:
                    all_articles.append([title_text, description_text, article_url])
                    seen_articles.add(article_url)

            return all_articles

        else:
            print(f"Erreur HTTP {response.status_code} lors de l'accès à {url}")
            return []

    except requests.RequestException as e:
        print(f"Erreur de requête pour {url}: {e}")
        return []

# Exemple d'utilisation
url = "https://www.lemonde.fr/economie/"
articles = scrape_all_page(url)
for article in articles:
    print(f"Titre : {article[0]}")
    print(f"Description : {article[1]}")
    print(f"URL : {article[2]}\n")
